# Setup

In [1]:
import pandas as pd
import numpy as np

Первый и второй xlsx

In [2]:
all_data = pd.DataFrame()
first = pd.read_excel('./first.xlsx')
second = pd.read_excel('./second.xlsx')
all_data = all_data.append(first,ignore_index=True)
all_data = pd.merge(all_data, second, how='outer')

Третий xlsx

In [3]:
third = pd.read_excel('./third.xlsx')

Переименуем названия столбцов с именами и id

In [4]:
all_data.rename(columns={'ФИО': 'name', 'Табельный  №': 'id'}, inplace=True)
third.rename(columns={'ФИО кандидата': 'name', 'таб.номер из БК': 'id'}, inplace=True)

Выделяем столбцы с именами

In [5]:
names = all_data['name'].dropna()
names.describe()

count         1767
unique        1551
top       Вакансия
freq           194
Name: name, dtype: object

In [6]:
names_diff = third['name'].dropna()
names_diff.describe()

count                         1731
unique                        1599
top       Турнаев Борис Викторович
freq                             2
Name: name, dtype: object

Аналогично с табелями

In [7]:
id_ = all_data['id'].dropna()
id_.astype(object).describe()

count         1572.0
unique        1572.0
top       25004031.0
freq             1.0
Name: id, dtype: float64

In [8]:
id_diff = third['id'].dropna()
id_diff.describe()

count         1115
unique        1081
top       17016737
freq             2
Name: id, dtype: object

# Работа с данными

## Имена

Убираем дубликаты

In [9]:
names = names.drop_duplicates(keep=False)
names_diff = names_diff.drop_duplicates(keep=False)

In [10]:
names.count()

1527

In [11]:
names_diff.count()

1467

Производим объединение обоих сетов и убираем дубликаты

In [12]:
difference_names = pd.concat([names,names_diff]).drop_duplicates(keep=False)

In [13]:
difference_names.count()

426

Итого: 426 уникальных имён

## Табельные номера

Убираем дубликаты

In [14]:
id_ = id_.drop_duplicates(keep=False)
id_diff = id_diff.drop_duplicates(keep=False)

In [15]:
id_.count()

1572

In [16]:
id_diff.count()

1047

Необходима стандартизация id, из-за их разного представления(str №123, float 123.0, int 123)

In [17]:
def fixed(x):
    if x.startswith('№'):
        return x[1:]
    
def fix_numbers(list_):
    #Убираем пустные значения и приводим все к строке
    list_ = list_.dropna().apply(str)
    #Убираем знак '№' в начале id, если он присутствует
    list_ = list_.apply(lambda x: fixed(x) if x.startswith('№') else x)
    
    #Cначала конвертируем во float, т.к str(123.0) нельзя сразу сконвертировать в int
    return list_.apply(float).apply(np.int64)

In [18]:
id_ = fix_numbers(id_)
id_diff = fix_numbers(id_diff)

In [19]:
print(id_.dtype)
print(id_diff.dtype)

int64
int64


Производим объединение обоих сетов и убираем дубликаты

In [20]:
difference_ids = pd.concat([id_,id_diff]).drop_duplicates(keep=False)
difference_ids.count()

1314

1314 уникальных id

## Имена и идентификаторы вместе

In [21]:
df = pd.concat([id_, names], axis=1)
df_diff = pd.concat([id_diff, names_diff], axis=1)

In [22]:
df_all = pd.merge(df, df_diff, how='inner').dropna()

In [23]:
df_all.count()

id      619
name    619
dtype: int64

Итого: 619 совпадений